In [20]:
from pathlib import Path
import shutil
import pandas as pd
import json

Outputfiles zusammenführen:

In [2]:
# with open(f"../data/processed/2_5_processed.jsonl", "wb") as f_out:
#     for chunk in Path("../data/processed/").glob("2_5_processed-*.jsonl"):
#         with open(chunk,'rb') as f_in:
#             shutil.copyfileobj(f_in, f_out)
#         chunk.unlink()

In [2]:
article_file = "../data/raw/1_tagesschau.jsonl"
articles = pd.read_json(open(article_file, "r", encoding="utf8"), lines=True)

entity_file = "../data/processed/2_5_processed.jsonl"
entities = pd.read_json(open(entity_file, "r", encoding="utf8"), lines=True)

In [3]:
articles['tstamp'] = articles['tstamp'].apply(lambda x: pd.Timestamp(x, unit='s', tz='Europe/Berlin'))

In [4]:
joined = entities.join(articles[['tstamp', 'url']].set_index('url'), on='url', validate='many_to_one')

In [5]:
joined

,url,ner_text,label,wd_id,wd_url,score,tstamp
0,/wirtschaft/venezuela-oel-101.html,Russland,LOC,Q159,https://www.wikidata.org/wiki/Q159,0.2023,2015-01-31 11:04:27+01:00
1,/wirtschaft/venezuela-oel-101.html,Nigeria,LOC,Q1033,https://www.wikidata.org/wiki/Q1033,0.3804,2015-01-31 11:04:27+01:00
2,/wirtschaft/venezuela-oel-101.html,Venezuela,LOC,Q717,https://www.wikidata.org/wiki/Q717,0.3854,2015-01-31 11:04:27+01:00
3,/wirtschaft/venezuela-oel-101.html,Caracas,LOC,Q1533,https://www.wikidata.org/wiki/Q1533,0.3827,2015-01-31 11:04:27+01:00
4,/wirtschaft/venezuela-oel-101.html,Venezuela,LOC,Q717,https://www.wikidata.org/wiki/Q717,0.3854,2015-01-31 11:04:27+01:00
...,...,...,...,...,...,...,...
1188418,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Deutschlands,LOC,Q43287,https://www.wikidata.org/wiki/Q43287,0.0876,2023-03-17 14:01:39+01:00
1188419,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,russischen,MISC,Q159,https://www.wikidata.org/wiki/Q159,0.3977,2023-03-17 14:01:39+01:00
1188420,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Angriffskriegs,MISC,Q110999040,https://www.wikidata.org/wiki/Q110999040,0.3167,2023-03-17 14:01:39+01:00
1188421,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Ukraine,LOC,Q212,https://www.wikidata.org/wiki/Q212,0.6691,2023-03-17 14:01:39+01:00


In [13]:
df = joined.drop_duplicates(['url', 'wd_id']).copy()

In [27]:
# Group by day and entity, then count occurrences
mentions_per_day = df.groupby([pd.Grouper(key='tstamp', freq='D'), 'wd_id', 'label']).size().reset_index(name='count')
mentions_per_day.sort_values(by='count')

,tstamp,wd_id,label,count
0,2015-01-01 00:00:00+01:00,Q1068361,ORG,1
209169,2022-11-17 00:00:00+01:00,Q7204,LOC,1
209172,2022-11-17 00:00:00+01:00,Q7275,LOC,1
209173,2022-11-17 00:00:00+01:00,Q7278,ORG,1
209174,2022-11-17 00:00:00+01:00,Q72845,PER,1
...,...,...,...,...
323057,2023-06-20 00:00:00+02:00,Q183,LOC,24
220779,2022-12-09 00:00:00+01:00,Q183,LOC,25
213319,2022-11-25 00:00:00+01:00,Q183,LOC,27
262529,2023-02-24 00:00:00+01:00,Q212,LOC,29


In [30]:
# Group by day and entity, then count occurrences
mentions_per_day = df.groupby([pd.Grouper(key='tstamp', freq='W'), 'wd_id', 'label']).size().reset_index(name='count')
mentions_per_day.sort_values(by='count')

,tstamp,wd_id,label,count
0,2015-01-04 00:00:00+01:00,Q100749,PER,1
142680,2022-10-23 00:00:00+02:00,Q114966440,ORG,1
142686,2022-10-23 00:00:00+02:00,Q115042233,PER,1
142687,2022-10-23 00:00:00+02:00,Q115052,PER,1
142688,2022-10-23 00:00:00+02:00,Q115400,PER,1
...,...,...,...,...
228975,2023-06-25 00:00:00+02:00,Q183,LOC,107
209246,2023-04-30 00:00:00+02:00,Q183,LOC,108
145795,2022-10-30 00:00:00+02:00,Q183,LOC,109
188097,2023-02-26 00:00:00+01:00,Q212,LOC,112


In [15]:
df.shape

(527860, 7)

In [16]:
df["url"].nunique()

24996

In [19]:
# Nicht disambiguierte und niedrig bewertete Entities entfernen
df.dropna(inplace=True)
df = df[df["score"] >= 0.1]
df.shape

(438678, 7)

In [23]:
pwd()

'/home/patrick/Dev/news_and_elections/notebooks'

In [24]:
with open(f"../data/processed/3_catalog.json", "w") as f_out:
    json.dump(df["wd_id"].sort_values().drop_duplicates().to_list(), f_out)

In [28]:
df

,url,ner_text,label,wd_id,wd_url,score,tstamp
0,/wirtschaft/venezuela-oel-101.html,Russland,LOC,Q159,https://www.wikidata.org/wiki/Q159,0.2023,2015-01-31 11:04:27+01:00
1,/wirtschaft/venezuela-oel-101.html,Nigeria,LOC,Q1033,https://www.wikidata.org/wiki/Q1033,0.3804,2015-01-31 11:04:27+01:00
2,/wirtschaft/venezuela-oel-101.html,Venezuela,LOC,Q717,https://www.wikidata.org/wiki/Q717,0.3854,2015-01-31 11:04:27+01:00
3,/wirtschaft/venezuela-oel-101.html,Caracas,LOC,Q1533,https://www.wikidata.org/wiki/Q1533,0.3827,2015-01-31 11:04:27+01:00
8,/wirtschaft/venezuela-oel-101.html,Nicolas Maduro,PER,Q58132,https://www.wikidata.org/wiki/Q58132,0.9917,2015-01-31 11:04:27+01:00
...,...,...,...,...,...,...,...
1188414,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,USA,LOC,Q30,https://www.wikidata.org/wiki/Q30,0.6348,2023-03-17 14:01:39+01:00
1188415,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Japan,LOC,Q17,https://www.wikidata.org/wiki/Q17,0.6340,2023-03-17 14:01:39+01:00
1188419,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,russischen,MISC,Q159,https://www.wikidata.org/wiki/Q159,0.3977,2023-03-17 14:01:39+01:00
1188420,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Angriffskriegs,MISC,Q110999040,https://www.wikidata.org/wiki/Q110999040,0.3167,2023-03-17 14:01:39+01:00


In [38]:
df_wd_entities = pd.read_json(open("../data/processed/4_wd-catalog.jsonl", "r", encoding="utf8"), lines=True)
df_wd_entities

,wd_url,wd_id,wd_label,wd_description,wd_redirect
0,https://www.wikidata.org/wiki/Special:EntityDa...,Q1000317,AOK Rheinland/Hamburg,organization,NaN
1,https://www.wikidata.org/wiki/Special:EntityDa...,Q1000053,Wassili Alexejewitsch Nebensja,Russian diplomat,NaN
2,https://www.wikidata.org/wiki/Special:EntityDa...,Q100026,General Dynamics F-16 Fighting Falcon,1974 multi-role fighter aircraft family by Gen...,NaN
3,https://www.wikidata.org/wiki/Special:EntityDa...,Q1,Universum,all of the spacetime and its contents includin...,NaN
4,https://www.wikidata.org/wiki/Special:EntityDa...,Q1000333,Huancavelica,"capital city of Huancavelica, Peru",NaN
...,...,...,...,...,...
45902,https://www.wikidata.org/wiki/Special:EntityDa...,Q9997,Borne,municipality and village in the Netherlands,NaN
45903,https://www.wikidata.org/wiki/Special:EntityDa...,Q99974,Mykorrhiza,symbiotic association between a fungus and the...,NaN
45904,https://www.wikidata.org/wiki/Special:EntityDa...,Q99937,Cesenatico,Italian comune,NaN
45905,https://www.wikidata.org/wiki/Special:EntityDa...,Q99950,Predappio,Italian comune,NaN


In [39]:
df

,url,ner_text,label,wd_id,wd_url,score,tstamp
0,/wirtschaft/venezuela-oel-101.html,Russland,LOC,Q159,https://www.wikidata.org/wiki/Q159,0.2023,2015-01-31 11:04:27+01:00
1,/wirtschaft/venezuela-oel-101.html,Nigeria,LOC,Q1033,https://www.wikidata.org/wiki/Q1033,0.3804,2015-01-31 11:04:27+01:00
2,/wirtschaft/venezuela-oel-101.html,Venezuela,LOC,Q717,https://www.wikidata.org/wiki/Q717,0.3854,2015-01-31 11:04:27+01:00
3,/wirtschaft/venezuela-oel-101.html,Caracas,LOC,Q1533,https://www.wikidata.org/wiki/Q1533,0.3827,2015-01-31 11:04:27+01:00
8,/wirtschaft/venezuela-oel-101.html,Nicolas Maduro,PER,Q58132,https://www.wikidata.org/wiki/Q58132,0.9917,2015-01-31 11:04:27+01:00
...,...,...,...,...,...,...,...
1188414,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,USA,LOC,Q30,https://www.wikidata.org/wiki/Q30,0.6348,2023-03-17 14:01:39+01:00
1188415,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Japan,LOC,Q17,https://www.wikidata.org/wiki/Q17,0.6340,2023-03-17 14:01:39+01:00
1188419,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,russischen,MISC,Q159,https://www.wikidata.org/wiki/Q159,0.3977,2023-03-17 14:01:39+01:00
1188420,/wirtschaft/weltwirtschaft/oecd-wachstum-langs...,Angriffskriegs,MISC,Q110999040,https://www.wikidata.org/wiki/Q110999040,0.3167,2023-03-17 14:01:39+01:00


In [41]:
df_final = df.join(df_wd_entities.loc[:, ['wd_id', 'wd_label', 'wd_description']].set_index('wd_id'), on='wd_id')

In [48]:
# Group by day and entity, then count occurrences
mentions_per_day = df_final.groupby([pd.Grouper(key='tstamp', freq='D'), 'wd_id', 'label', 'wd_label', 'wd_description']).size().reset_index(name='count')
mentions_per_day.sort_values(by='tstamp')
mentions_per_day.to_json("../data/processed/3_5_mentions_per_day.jsonl", orient="records", date_format='iso', lines=True)

In [52]:
# Group by week and entity, then count occurrences
mentions_per_week = df_final.groupby([pd.Grouper(key='tstamp', freq='W'), 'wd_id', 'label', 'wd_label', 'wd_description']).size().reset_index(name='count')
mentions_per_week.sort_values(by='count', ascending=False, inplace=True)
mentions_per_week.to_json("../data/processed/3_5_mentions_per_week.json", orient="records", date_format='iso', indent=2)
mentions_per_week.to_json("../data/processed/3_5_mentions_per_week.jsonl", orient="records", date_format='iso', lines=True)